# Quick Start

This notebook serves as a tutorial and illustrates how to use Path4GMNS step by step.

## 1. Install Path4GMNS

In [ ]:
pip install path4gmns

## 2. Prepare Input Data

node.csv and link.csv complying GMNS is the minimum requirement for you to get started. You can either download the sample data sets with the built-in function or prepare your own data set.

In [ ]:
import path4gmns as pg

download_sample_data_sets() will download five sample data sets including ASU, Braess Paradox, Chicago Sketch, Lima Network, Sioux Falls, and Two Corridors. Each of them contains the following files.

1. node.csv
2. link.csv
3. demand.csv
4. settings.yml (for multimodal analyses)
5. settings.csv (for DTALite)

In [ ]:
pg.download_sample_data_sets()

Navigate to the target data set directory. In this tutorial, we will use the [data set](https://github.com/jdlph/Path4GMNS/tree/dev/tests) where this Jupyter notebook is located for simplicity. You can check your present working directory (PWD) by the following commnad.

In [ ]:
pwd

If you are working in an online environment, e.g., Google Colab. You can move to one of the downloaded sample data sets, say ASU, via the following lines. 

In [ ]:
import os
os.chdir('data/ASU')

## 3. The Basic Functionalities
### 3.1 Get the Shortest Path between Two Nodes
Find the (static) shortest path (based on distance) and output it in the format of a sequence of node/link IDs.

In [ ]:
# create the network object by reading node.csv and link.csv
network = pg.read_network()

# node path from node 1 to node 2
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2))
# link path from node 1 to node 2
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link'))

Path4GMNS uses **mile** and **mile per hour (mph)** as the default units for length and speed in its internal calculation and all outputs. If your link.csv features different length and speed units, please specify them in read_network(). The supported units are list as blow.

 **Length Units**
 * meter and m
 * kilometer and km
 * mile and mi

 **Speed Units**
 * kmh or kph
 * mph
 
 You would have to do manual conversion if your units are not among the above ones.

In [ ]:
# the default are length_unit='mile' and speed_unit='mph'
network = pg.read_network(length_unit='meter', speed_unit='kph')

# node path from node 1 to node 2
print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2))
# link path from node 1 to node 2
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, seq_type='link'))

### 3.2 Synthesize Zones and OD Demand Matrix
Zone information is crucial in conducting traffic assignment, evaluating accessibility and equity. When no zone information is provided along node.csv, Path4GMNS can synthesize zones as well as OD demand matrix for you. Note that it will **OVERWRITE** any existing zone.csv and demand.csv if you have them.

In [ ]:
network = pg.read_network()

# by default, grid_dimension is 8, total_demand is 10,000,
# time_budget is 120 min, mode is 'auto'
pg.network_to_zones(network)
pg.output_zones(network)
# synthesized demand will be saved as demand.csv 
# and will overwrite any existing demand file with the same name.
# synthesized zone will be output as zone.csv.
pg.output_synthesized_demand(network)

If you have zone.csv and demand.csv, you can skip this above step and load the synthesized zones and OD demand matrix using the following commands.

In [ ]:
network = pg.read_network()

# it reads zone.csv by default
pg.read_zones(network)
# it reads demand.csv by default
pg.load_demand(network)

### 3.3 Perform Path-Based UE Traffic Assignment

OD demand matrix is essential to perform this functionality. You will need to load the demand via pg.load_demand(network) before you can conduct traffic assignment via the following lines.

In [ ]:
network = pg.read_network()
pg.read_zones(network)
pg.load_demand(network)

# specify the parameters for traffic assignment
column_gen_num = 10
column_update_num = 10

# path-based UE only
pg.perform_column_generation(column_gen_num, column_update_num, network)

# if you do not want to include geometry info in the output file,
# use pg.output_columns(network, False)
# output column information to agent.csv
pg.output_columns(network)
# output link performance to link_performance.csv
pg.output_link_performance(network)

### 3.4 Evaluate Accessibility

Accessiblity defines where you can go given a time budget and a transportation mode (e.g., auto). You can find the number of accessible zones from each zone (zone_accessibility.csv) along with the free flow travel time for each OD pair **in minutes** (od_accessibility.csv).

The default mode is 'auto' and the default time budget is 240 minutes. We will come back to multimodal evaluation in a later section. Zone information is necessary for accessibility evaluation.

In [ ]:
# no need to load demand file for accessibility evaluation
network = pg.read_network()
pg.read_zones(network)

pg.evaluate_accessibility(network, single_mode=True)

### 3.5 Evaluate Equity

Transportation equity is accessibility with respect to different demographics. Path4GMNS rovides the following simple info and statistics on equity given a time budget and a segmentation of zones.

1. accessible zones.
2. min accessibility. This metric refers to the zone with the minimum number of accessible zones. 
3. max accessibility. This metric refers to the zone with the maximum number of accessible zones. 
4. mean accessibility. The average number of accessible zones over a bin of zones (corresponding to a specific demographic) given a time budget and a transportation mode.

Similar to accessiblity evaluation, the default mode is 'auto' but the default time budget is 60 minutes. Zone information is still required here.

In [ ]:
# no need to load demand file for equity evaluation
network = pg.read_network()
pg.read_zones(network)

pg.evaluate_equity(network, single_mode=True)

## 4. Move Foward to Multimodal Evaluation

In order to perform multimodal evaluation, the corresponding modes (i.e., agent types) must be presented in settings.yml. It will be parsed by pyyaml (5.1 or higher) to the Python engine at run-time. 

pyyaml must be installed before running any multimodal evaluation.

In [ ]:
pip install pyyaml

### 4.1 Sample setting.yml
A sample file looks like blow. You can start from here and modify it towards your own needs. **Note that** the default agent type which is **'a' as type and 'auto' as name must be present in this file**. Otherwise, you will encounter an exception with message "No AGENT type: a". We will relax this requirement in a later release.

```yaml
agents:
  - type: a
    name: auto
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 60
    use_link_ffs: true
  - type: w
    name: walk
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 10
    use_link_ffs: false

demand_periods:
  - period: AM
    time_period: 0700-0800

demand_files:
  - file_name: demand.csv
    period: AM
    agent_type: a
```

You can download a sample settings.yml (to your PWD) if you do not have one by the following command.

In [ ]:
pg.download_sample_setting_file()

### 4.2 Get the Shortest Path between Two Nodes under a Specific Mode
In 3.1, we used this functionlity implicitly for the default mode, which is 'a' or equivalently 'auto'. Now with 'walk' defined in settings.yml, we are able to find the shortest path under mode 'w' or 'walk'.

In [ ]:
network = pg.read_network()

print('\nshortest path (node id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w'))
print('\nshortest path (link id) from node 1 to node 2, '
      +network.find_shortest_path(1, 2, mode='w', seq_type='link'))

For this test data set, 3.1 and 4.1 have the same results as each link is open to all modes, i.e., their "allowed_uses" are "all". 

### 4.3 Perform Multimodal Accessibility Evaluation

It is very similar to what we have done in 3.4 for a single-mode analysis.

In [ ]:
# no need to load demand file for accessibility evaluation
network = pg.read_network()
pg.read_zones(network)

pg.evaluate_accessibility(network)

### 4.4 Perform Multimodal Equity Evaluation

By removing "single_mode=True" from the arugment list in evaluate_equity(), multimodal evaluation will be triggered for all modes defined in settings.yml

In [ ]:
# no need to load demand file for equity evaluation
network = pg.read_network()
pg.read_zones(network)

pg.evaluate_equity(network)

### 4.5 Conduct Dynamic Traffic Simulation

Traffic simulation requires routing decision for each agent, which comes from the UE traffic assignment.

In [ ]:
network = pg.read_network()

pg.read_zones(network)
pg.load_demand(network)

# UE + DTA
column_gen_num = 10
column_update_num = 10
pg.perform_column_generation(column_gen_num, column_update_num, network)
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)

If you have agent.csv (i.e., columns) from a previous run or DTALite, you can bypass perform_column_generation() and directly load it to conduct simulation.

In [ ]:

import path4gmns as pg

network = pg.read_network()

# load existing UE result
pg.load_columns(network)
# DTA
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)


If you are still interested in traffic simulation using shortest paths, it can be achieved by setting column_gen_num as 1 and column_update_num as 0 illustrated below.

In [ ]:
import path4gmns as pg

network = pg.read_network()
pg.read_zones(network)
pg.load_demand(network)

# the following setting will set up the shortest path for each agent
column_gen_num = 1
column_update_num = 0
pg.perform_column_generation(column_gen_num, column_update_num, network)
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)

## 5. Advanced Features

### 5.1 In Case of Special Events

A special event often comes with capacity reduction over affected links. You can introduce one special event for each demand period in settings.yml as below. 

```yaml
demand_periods:
  - period: AM
    time_period: 0700-0800
    special_event:
      name: work_zone
      enable: true
      affected_links:
        - link_id: 1
          capacity_ratio: 0.5
        - link_id: 2
          capacity_ratio: 0.4
        - link_id: 3
          capacity_ratio: 0.6
        - link_id: 4
          capacity_ratio: 0
```

If the original capacity of an affected link i is C, its capacity then will be r * C with a reduction ratio of r when a special event is present. For an affected link, setting its capacity_ratio to 0 is equivalent to removing it from the entire demand period. You can turn on or off a special event by setting enable to true or false.

Then you can conduct traffic assignment (as shown in 3.3) and see the impact from a special event.

### 5.2 Accessibility Considering Time-Dependent Link Travel Time

In 3.4 and 4.4, accessiblity is evluated using the link free flow travel time, which is determined by link length and link free-flow speed under a specific mode. 

Link travel time varies over time so does accessibility. When the time-dependent accessibility is of interested, time-dependent link travel time (i.e., VDF_fftt from a given demand period in link.csv) will come into play by overwriting the (static) link free-flow speed (from either link.csv or settings.yml. both are denoted as "free_speed").

In [ ]:
network = pg.read_network()

# time-dependent accessibility under the default mode auto (i.e., 'a')
# for demand period 0 (i.e., VDF_fftt1 in link.csv will be used in the evaluation)
pg.evaluate_accessibility(network, single_mode=True, time_dependent=True)

# if you would like to evaluate accessibility under a target mode, say walk, then
pg.evaluate_accessibility(network, single_mode=True, mode='w', time_dependent=True)

## 6. DTALite

Path4GMNS also serves as an API to the C++-based DTALite to conduct various multimodal traffic assignments including,

    0: Link-based UE,
    1: Path-based UE,
    2: UE + Dynamic Traffic Assignment (DTA),
    3: OD Matrix Estimation (ODME).

Here we only demonstrate how to perform path-based UE (i.e., mode 1) using DTALite from Path4GMNS. You would need [settings.csv](https://github.com/jdlph/Path4GMNS/blob/dev/tests/settings.csv) to get started.

In order to use DTALite embedded in Path4GMNS, run-time support OpenMP must be installed in the first space. See [Installation](https://path4gmns.readthedocs.io/en/latest/installation.html) for details.

In [ ]:
mode = 1
column_gen_num = 10
column_update_num = 10

pg.perform_network_assignment_DTALite(mode, column_gen_num, column_update_num)

Starting from v0.9.6, you could conduct the above tasks using the [Multimodal DTALite](https://github.com/asu-trans-ai-lab/DTALite). No parameters are needed.

In [ ]:
pg.run_DTALite()

Please consider running the script calling this API using system terminal rather than the Python console for proper DTA logging.

## 7. Legacy Way to Load Demand and Zone Info

### A Little Bit of History

During the early development of Path4GMNS, demand.csv is presumed to be given and zone information is provided along node.csv. The latter is the current design of GMNS and is endorsed in [OSM2GMNS](https://github.com/jiawlu/OSM2GMNS). This assumption leads to the following legacy way (as the original design of Path4GMNS) to load demand and zone information while the new way demonstrated was in **Section 3.2**. 


```python
network = pg.read_network(load_demand=True)
```


The demand file is specified in settings.yml as demand.csv. You can change it to any demand file in your PWD. This design actually allows us to load multiple demand files simultaneously corresponding to different period and agent_type. We will elaborate it in a future release.

```yaml
---
# settings for Path4GMNS
agents:
  - type: a
    name: auto
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 60
    use_link_ffs: true
  - type: w
    name: walk
    vot: 10
    flow_type: 0
    pce: 1
    free_speed: 10
    use_link_ffs: false

demand_periods:
  - period: AM
    time_period: 0700-0800

demand_files:
  - file_name: demand.csv
    period: AM
    agent_type: a

```
If you came to Path4GMNS before v0.8.6, you probably notice that "load_demand=True" was not required as demand will be loaded by default. This is one of the changes introduced in v0.8.6 to better promote the new way of loading demand. The legacy way are still used in [demo.py](https://github.com/jdlph/Path4GMNS/blob/dev/tests/demo.py) and the code examples in [User Manual](https://github.com/jdlph/Path4GMNS/tree/dev#user-manual), which will be updated gradually over time. You can still use this legacy way with the [existing sample data sets](https://github.com/jdlph/Path4GMNS/tree/dev/data), which do not have zone.csv but have zone information in node.csv. 

**Note that** read_network(load_demand=True) will **automatically check OD connectivity** which **requires zone info to be specified in node.csv**. Otherwise, you will get an exception stating "NO VALID OD VOLUME!! DOUBLE CHECK YOUR demand.csv". 

### 7.1 Perform Path-Based UE Traffic Assignment

In [ ]:
# zone info must be present in node.csv
# demand.csv must be given
network = pg.read_network(load_demand=True)

column_gen_num = 10
column_update_num = 10

# path-based UE only
pg.perform_column_generation(column_gen_num, column_update_num, network)

# if you do not want to include geometry info in the output file,
# use pg.output_columns(network, False)
pg.output_columns(network)
pg.output_link_performance(network)

### 7.2 Evaluate Multimodal Accessibility

In [ ]:
# zone info must be present in node.csv
# no need to load demand file for accessibility evaluation
network = pg.read_network()

pg.evaluate_accessibility(network)

### 7.3 Evaluate Multimodal Equity

In [ ]:
# zone info must be present in node.csv
# no need to load demand file for equity evaluation
network = pg.read_network()

pg.evaluate_equity(network)

### 7.4 Conduct Dynamic Traffic Simulation

In [ ]:
network = pg.read_network(load_demand=True)

column_gen_num = 10
column_update_num = 10
pg.perform_column_generation(column_gen_num, column_update_num, network)
pg.perform_simple_simulation(network)
print('complete dynamic simulation.\n')

print('writing agent trajectories')
pg.output_agent_trajectory(network)